# MIDAS_PYTHON_TASK
The following notebook contains solution to python task given to applicants for summer intern position at MIDAS IIITD Lab.The notebook is made up of 3 parts.
<ol>
    <li>Import Dependencies</li>
    <li>Mine tweets and store them in a JSONLines file</li>
    <li>Read from JSONLines file and display few attributes in a tabular way</li>
</ol>

### Part 1:Import Dependencies
We impart all the neccessary dependencies like tweepy,Twitter's python API to fetch tweets and jsonlines,which is required to write tweets in json form into a jsonl file.

In [2]:
import tweepy
from tweepy import OAuthHandler
import jsonlines
import json

### Part 2: Mine tweets and store them in a JSONLines file
In order to mine tweets from Twitter we first need to create tweepy's API object with the help of various keys that twitter gives to the developer.This object helps you fetch tweets from twitter according to your needs.

In [3]:
##KEYS 
##NOT DISCLOSED COZ OF PRIVACY
consumer_key="liABpG2XM62tIo02fjen7FDVJ"
consumer_secret="nUI6fH6PJuuUvQFlzylCieFxOZ8VZdf5yGrRGK294whWEenayA"
access_token="1046393582469500928-a4Il71HUPkyF0MtoS3V6dJL1igx84k"
access_secret="
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

Now that the object has been created we use it to extract tweet from @midasIIITD's twitter handles.Twitter returns all the tweets in the form of a status object.We convert these into json and then store them in a list.We use the api.user_timeline feature and mine the recent 200 tweets in the "extended" mode.

In [5]:
def user_tweets(api,user):
    #for tweet in tweepy.Cursor(api.user_timeline(screen_name=user,count=200)).items(10):
    #    print(tweet)
    new_tweets = api.user_timeline(screen_name=user, count=200, tweet_mode="extended")
    tweet_strs = []
    for tweet in new_tweets:
        tweet_strs.append(tweet._json)
    return tweet_strs

tweets = user_tweets(api,'midasIIITD')

We now take the list and write into a jsonlines file.So that they can be reused again,All the tweets are stored in json format in tweets.jsonl file.

In [6]:
with jsonlines.open('tweets.jsonl', mode='w') as writer:
    writer.write(tweets)

### Part 3 : Read from JSONLines file and display few attributes in a tabular way
We first read the .jsonl file and then store it in a list so that further preprocessing can be done in a meaningful manner.

In [7]:
list_tweets = []
with jsonlines.open('tweets.jsonl') as reader:
    for obj in reader:
        list_tweets = obj

We need to extract 5 details for each tweet.The details are .
<ol>
    <li>The text of the tweet.</li>
    <li>Date and time of the tweet.</li>
    <li>The number of favorites/likes.</li>
    <li>The number of retweets.</li>
    <li>Number of Images present in Tweet.If no image returns None.</li>
</ol>
The following 5 functions help in doing that and are given in their natural order.

### Note : In order to extract images we look if there is 'media' key in the json.If there is a media key,we look at the objects in this field.If any objects 

In [8]:
#Fxn to extract the full tweets
#Sometimes tweets are truncated(the first if condition checks for that) we then use a different method to retrieve the full text tweet by having a look at if it was retweeted or not.
def extract_full_tweets(tweet):
    if tweet['full_text'][-1:] == '…':
        list_of_texts = tweet['retweeted_status']['full_text'].split('\n')
        tweet_text = ''.join(list_of_texts)
    else :
        list_of_texts = tweet['full_text'].split('\n')
        tweet_text = ''.join(list_of_texts)
    return tweet_text

#FXn to extract Date and Time of the tweet
def extract_date_and_time(tweet):
    return tweet['created_at']

#Fxn to extract likes of each time
def extract_likes(tweet):
    return tweet['favorite_count']

#Fxn to extract retweet count
def extract_retweet_count(tweet):
    return tweet['retweet_count']
                 
#Fxn to extract no.of images
#We check if any media exists in the tweet json and then we iterate through each media object and if it is a phot we increment the counter variable by 1
def extract_num_images(tweet):
    if 'media' in tweet['entities'].keys():
        counter = 0
        for media_objects in tweet['entities']['media']:
            if media_objects['type'] == 'photo':
                counter+=1
        if counter:
            return str(counter)
        else:
            return "None"
    else:
        return "None"
    return 0
    

Now we iterate through all these tweets and retrieve 5 features for these tweets and then create a list of lists containg all these features for all the tweets.

In [9]:
list_of_lists = []
for tweet in list_tweets:
    list_of_lists.append([extract_full_tweets(tweet),extract_date_and_time(tweet),extract_likes(tweet),extract_retweet_count(tweet),extract_num_images(tweet)])  

#One Row 
list_of_lists[8]

['Last day to register for #PortfolioCreationinDesign Workshop Indraprastha Institute of Information Technology, Delhi on 15th March by Ankur SardanaRegister at - https://t.co/LlO29aGmaE #HCDIIITD #workshop https://t.co/EWQ7fuGNDD',
 'Wed Mar 13 17:09:44 +0000 2019',
 0,
 2,
 'None']

We now convert the list of lists into a pandas dataframe.So that it can be displayed in a nice tabular form.For convenience I have displayed only the first 30 rows because of space constraints.

In [10]:
import pandas as pd
df = pd.DataFrame(list_of_lists)
df.columns = ["Tweet_text", "Date,Time,Year", "No.of likes", "No.of Retweets","No.of Images"]
pd.set_option('display.max_colwidth', -1)
#display first 30 tweets
df.head(30)

,Tweet_text,"Date,Time,Year",No.of likes,No.of Retweets,No.of Images
0,@IEEEBigMM19 is also available on Facebook now. LIKE its Facebook page https://t.co/B3Q0zmmzXb to get the regular updates. Check more details at https://t.co/w9ZymoPisk #IEEE #BigMM19 #Big #Multimedia #Singapore,Wed Mar 20 08:19:24 +0000 2019,1,1,None
1,"BigMM 2019 : IEEE BigMM 2019 – Call for Workshop Proposals https://t.co/I4vqf8FE6K … When: Sep 11, 2019 - Sep 13, 2019 Where: SingaporeSubmission Deadline: Apr 1, 2019 Notification Due: Apr 10, 2019 #IEEE #BigData #Singapore #Multimedia",Wed Mar 20 02:40:07 +0000 2019,0,4,None
2,"BigMM 2019 : IEEE BigMM 2019 – Call for Workshop Proposalshttps://t.co/oUq2G0UgKNWhen: Sep 11, 2019 - Sep 13, 2019Where: SingaporeSubmission Deadline: Apr 1, 2019Notification Due: Apr 10, 2019#IEEE #BigMM #Workshop #Proposal #Singapore #Multimedia",Mon Mar 18 02:27:47 +0000 2019,6,3,None
3,"Congratulations @midasIIITD team, Rohan, Pradyumn, Ramit, @debanjanbhucs, @MADAIguy and @RatnRajiv for getting their paper titled, ""SNAP-BATNET: Cascading Author Profiling and Social Network Graphs for Suicide Ideation Detection on Social Media"", accepted at @NAACLHLT SRW.",Sun Mar 17 14:22:04 +0000 2019,15,4,None
4,We have emailed the task details to all shortlisted candidates for @midasIIITD internship. Kindly check the same. If you are one of the shortlisted candidates and have not received the email then check your SPAM first before approaching. #MIDAS #Summer #Research #Internship,Sat Mar 16 14:06:56 +0000 2019,6,0,None
5,"IEEE BigMM 2019 - Call for Workshop Proposals. Contact @midasIIITD @RatnRajiv @debanjanbhucs if you have any query. Deadlines: - Proposals due: April 1, 2019- Acceptance notification: April 10, 2019Further information:- Webpage: https://t.co/5XX5Wyxp5T#IEEE #BigMM19",Sat Mar 16 09:20:29 +0000 2019,1,1,None
6,"Congratulations! Arijit, Ramit, @debanjanbhucs @RatnRajiv for your @NAACLHLT @naacl SRW 2019 paper, ""Speak Up, Fight Back! Detection of Social Media Disclosures of Sexual Harassment"". Check more details at https://t.co/i6eAHg7gcN#MeToo #Student #Research #Paper #NAACL",Sat Mar 16 09:14:58 +0000 2019,7,2,None
7,We will be releasing a very interesting task to shortlisted candidates for the internship at @midasIIITD today. You will be having 10 days to solve the same. You will get a detailed email on the task and submission instruction today. Looking forward to your out of box solution.,Sat Mar 16 05:13:14 +0000 2019,7,2,None
8,"Last day to register for #PortfolioCreationinDesign Workshop Indraprastha Institute of Information Technology, Delhi on 15th March by Ankur SardanaRegister at - https://t.co/LlO29aGmaE #HCDIIITD #workshop https://t.co/EWQ7fuGNDD",Wed Mar 13 17:09:44 +0000 2019,0,2,None
9,"@ACMMM19 @sigmm @TheOfficialACM @acmmmsys @ACMTVX @ACMICMR @euromm @Benoit_Huet @RatnRajiv @kelkalot We need to clearly mention abstract and final submission deadlines at https://t.co/U7DTNNyw87Currently, we have just mentioned 1st April as the regular paper submission deadline. Kindly update. #ACMMM19 #Nice #Multimedia https://t.co/ocOCrlFHLH",Wed Mar 13 04:11:24 +0000 2019,1,0,1


Let us store these tweets in a csv file so that people can have a look at them afterwards. 

In [11]:
df.to_csv("MIDAS_TWEETS.csv",index=False)